### 라이브러리

In [115]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [116]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [117]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [118]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [119]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [120]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [121]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [122]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [123]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

In [124]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [125]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

In [126]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 1월

train['석식계'][244] = 398 ###
train['석식계'][492] = 398 ###
train['석식계'][730] = 398 ###
train['석식계'][973] = 398 ###


train['석식계'][281] = 398
# train['석식계'][502] = 478.8605851979346
train['석식계'][510] = 398
train['석식계'][529] = 398
train['석식계'][747] = 398
train['석식계'][766] = 398
train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

In [127]:
train.query('석식계 == 0 & 공휴일전후_0 == 0')

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,주,식사가능자수,월(중식),월(석식),공휴일전후_0,공휴일전후_1,공휴일전후_2
224,2016-12-28,2,2705,166,225,0,0.0,767.0,0.0,4,2016,12,28,52,2539.0,12,12,0,1,0
502,2018-02-14,2,2707,418,159,0,0.0,850.0,0.0,4,2018,2,14,7,2289.0,1,2,0,0,1
955,2019-12-31,1,3111,709,149,22,0.0,349.0,0.0,2,2019,12,31,1,2402.0,12,12,0,1,0


In [128]:
# 자기개발의날 라벨링

# train['자기개발의날아닌수'] = 0
# test['자기개발의날아닌수'] = 0

# train.loc[((train.석식계!=0) & (train.요일 == 2)), '자기개발의날아닌수'] = 1 ## 자기개발인날은 1 아닌날 0. 1월을 올리기위함

# train = pd.get_dummies(train, columns=['자기개발의날아닌수'])


# test.loc[(train.요일 == 2), '자기개발의날아닌수'] = 1 ## 자기개발인날은 1 아닌날 0. 1월을 올리기위함
# test = pd.get_dummies(test, columns=['자기개발의날아닌수'])

### 최종제출 ( 자기개발1대체,금)

In [129]:
## train.loc[(train.요일==4), '석식계'] -= 10

In [130]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [131]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [132]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7657572	total: 3.46ms	remaining: 3.46s
1:	learn: 198.8397145	total: 5.99ms	remaining: 2.99s
2:	learn: 193.8449503	total: 8.49ms	remaining: 2.82s
3:	learn: 189.4059077	total: 10.3ms	remaining: 2.57s
4:	learn: 185.2646636	total: 15.8ms	remaining: 3.14s
5:	learn: 181.0579138	total: 18.2ms	remaining: 3.02s
6:	learn: 176.9944339	total: 20.7ms	remaining: 2.94s
7:	learn: 173.0343166	total: 23.8ms	remaining: 2.96s
8:	learn: 169.2847999	total: 25.6ms	remaining: 2.82s
9:	learn: 165.7634692	total: 27.2ms	remaining: 2.69s
10:	learn: 162.3163108	total: 28.9ms	remaining: 2.59s
11:	learn: 158.9027017	total: 30.5ms	remaining: 2.51s
12:	learn: 156.0506426	total: 32.2ms	remaining: 2.44s
13:	learn: 153.5771069	total: 34.4ms	remaining: 2.42s
14:	learn: 150.7610607	total: 36.4ms	remaining: 2.39s
15:	learn: 148.0510638	total: 38ms	remaining: 2.34s
16:	learn: 145.6633184	total: 39.6ms	remaining: 2.29s
17:	learn: 143.4048205	total: 41.2ms	remaining: 2.25s
18:	learn:

In [133]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1044.487287,287.974971
1,2021-01-28,975.995475,510.550478
2,2021-01-29,583.311034,215.204132


In [134]:
submission

,일자,중식계,석식계
0,2021-01-27,1044.487287,287.974971
1,2021-01-28,975.995475,510.550478
2,2021-01-29,583.311034,215.204132
3,2021-02-01,1221.960499,508.840561
4,2021-02-02,1030.079119,558.917473
5,2021-02-03,1064.205208,509.505770
6,2021-02-04,1008.477227,556.388591
7,2021-02-05,729.449757,343.145595
8,2021-02-08,1292.217083,691.803925
9,2021-02-09,1086.126859,589.776938


In [113]:
submission['석식계'].mean()

532.0417242688542

In [61]:
submission.to_csv('자기개발1대체,라벨.csv', index=False)

In [234]:
train.query('자기개발의날아닌수_1==1').mean()

요일                   2.000000
본사정원수             2807.881773
본사휴가자수             111.709360
본사출장자수             238.305419
본사시간외근무명령서승인건수       9.605911
현본사소속재택근무자수         51.842365
중식계                910.788177
석식계                437.534199
요일(석식)               4.000000
년                 2018.059113
월                    6.433498
일                   14.098522
주                   26.098522
식사가능자수            2644.330049
월(중식)                6.571429
월(석식)                6.714286
공휴일전후_0              1.000000
공휴일전후_1              0.000000
공휴일전후_2              0.000000
자기개발의날아닌수_0          0.000000
자기개발의날아닌수_1          1.000000
dtype: float64